<a href="https://colab.research.google.com/github/zero0914/1091-1N-demo-208417146/blob/main/HW01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Drive
!gdown --id '1lIvskEvcypn344lZ2u_rT7XDwjn_zhjQ' --output food-11.zip

!unzip -q food-11.zip

Downloading...
From: https://drive.google.com/uc?id=1lIvskEvcypn344lZ2u_rT7XDwjn_zhjQ
To: /content/food-11.zip
963MB [00:11, 82.6MB/s]


In [ ]:
# Import necessary packages.
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader
from torchvision.datasets import DatasetFolder
from tqdm.auto import tqdm

In [ ]:

train_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])


In [ ]:
batch_size = 128

train_set = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
valid_set = DatasetFolder("food-11/validation", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
test_set = DatasetFolder("food-11/validation", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(4, 4, 0),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(256 * 8 * 8, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 11)
        )

    def forward(self, x):
        x = self.cnn_layers(x)

        x = x.flatten(1)

        x = self.fc_layers(x)
        return x

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"


model = Classifier().to(device)
model.device = device


criterion = nn.CrossEntropyLoss()


optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)

n_epochs = 10


for epoch in range(n_epochs):

    # ---------- Training ----------
    model.train()

    train_loss = []
    train_accs = []

    for batch in tqdm(train_loader):

        imgs, labels = batch


        logits = model(imgs.to(device))

        loss = criterion(logits, labels.to(device))

        optimizer.zero_grad()

        loss.backward()

        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        optimizer.step()

        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        train_loss.append(loss.item())
        train_accs.append(acc)

    
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")


    model.eval()


    valid_loss = []
    valid_accs = []


    for batch in tqdm(valid_loader):


        imgs, labels = batch


        with torch.no_grad():
          logits = model(imgs.to(device))


        loss = criterion(logits, labels.to(device))


        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        valid_loss.append(loss.item())
        valid_accs.append(acc)

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



[ Train | 001/010 ] loss = 2.29317, acc = 0.20687



[ Valid | 001/010 ] loss = 2.63037, acc = 0.12396



[ Train | 002/010 ] loss = 1.97329, acc = 0.29812



[ Valid | 002/010 ] loss = 2.12226, acc = 0.21641


...


[ Valid | 008/010 ] loss = 1.81545, acc = 0.38958



[ Train | 009/010 ] loss = 1.09281, acc = 0.63594



[ Valid | 009/010 ] loss = 2.03458, acc = 0.34714



[ Train | 010/010 ] loss = 1.01168, acc = 0.66125



[ Valid | 010/010 ] loss = 1.73157, acc = 0.38984


In [ ]:
model.eval()

predictions = []

for batch in tqdm(test_loader):

    imgs, labels = batch

    with torch.no_grad():
        logits = model(imgs.to(device))

    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

In [ ]:
from sklearn import metrics
target_names=["labels_0"]
for i in range(1,11):
    target_names.append('labels_'+str(i))
print("classification report:")
print(metrics.classification_report(test_set.targets,predictions,target_names=target_names,digits=4))
print("Accuracy: "+str(format(metrics.accuracy_score(test_set.targets,predictions),'0.4f')))
print("Confusion Matrix:")
print (metrics.confusion_matrix(test_set.targets,predictions))

classification report:
              precision    recall  f1-score   support

    labels_0     0.1875    0.1000    0.1304        60
    labels_1     0.3763    0.5833    0.4575        60
    labels_2     0.4545    0.1667    0.2439        60
    labels_3     0.3333    0.1333    0.1905        60
    labels_4     0.2167    0.7333    0.3346        60
    labels_5     0.4444    0.4000    0.4211        60
    labels_6     0.6000    0.4000    0.4800        60
    labels_7     0.6250    0.3333    0.4348        60
    labels_8     0.4267    0.5333    0.4741        60
    labels_9     0.6129    0.3167    0.4176        60
   labels_10     0.8333    0.7500    0.7895        60

    accuracy                         0.4045       660
   macro avg     0.4646    0.4045    0.3976       660
weighted avg     0.4646    0.4045    0.3976       660

Accuracy: 0.4045
Confusion Matrix:
[[ 6  8  1  4 30  3  4  0  1  1  2]
 [ 2 35  2  3 10  0  0  1  3  3  1]
 [ 1  9 10  2 16 11  1  0  7  3  0]
 [ 2 10  0  8 27  1  